In [2]:
import CoilcraftRandomSelect as crs
import MurataRandomSelect as mrs

print(crs.random_coilcraft_inductor()['SubCkt'])
print(mrs.random_murata_capacitor('C1')['SubCkt'])

*======================================================================
* SPICE Model generated by Coilcraft
* Coilcraft Part Number : Inductor
* Inductance = 150uH
*======================================================================
* Model Parameters:
*   Valid Frequency Range = 0.1MHz-3.5MHz
*   Ambient Temperature = 25 degC
*   Inductor Impedance Model
*   Use model for Time Domain simulations
*======================================================================
.subckt Inductor port1 port2 PARAMS: Cpar=5.69pF Ind=150uH
X1 port1 port2 Model1D PARAMS:
+ R1=462
+ R2=6.68
+ C= {Cpar}
+ K1=0.000291
+ K2=123
+ K3= {Ind}
+ K4=0
+ K5=0
+ L=0
+ Is=0
+ a=0
+ L_Z0=0
+ L_EL=0
+ L_F0=0E6
+ PkZ=47734.9309
.ends Inductor
*----------------------------------------------------------------------
* LTspice Model generated by Murata Manufacturing Co., Ltd.
* Copyright(C) Murata Manufacturing Co., Ltd.
* Description :0402M(01005)/C0G(-55to125[deg])/3.30e-06[uF]/16[V]
* Murata P/N :C1
* Property : 

In [38]:
from pathlib import Path
from LTTraceData import LTTraceData
from PyLTSpice import SimCommander, RawRead, SimRunner, SpiceEditor

high_side = True
low_side = False


def frequencytoltpulse(frequency, side: bool):
    deadband = 8  # ns
    period = 1 / (frequency / 1E9)  # Convert to Gigahertz to ensure the period will be in nanoseconds
    if side == low_side:
        return f"PULSE(3.3 0 0n 1n 1n {period + (deadband * 2)}n {period * 2}n)"
    else:
        return f"PULSE(0 3.3 {deadband}n 1n 1n {period}n {period * 2}n)"


fsw = 100E3
L = 10E-6
C = 47E-6

# select spice model
# LTC = SimCommander("LTFiles/EPC23102_Mine.asc")

v_out_max = 999
v_out_min = 0

while v_out_max > 6 or v_out_min < 4:
    LTC = SimRunner(output_folder='LTFiles')
    # LTC.create_netlist('LTFiles/EPC23102_Mine.asc')
    netlist = SpiceEditor('LTFiles/EPC23102_Mine.asc')
    # set default arguments
    netlist.set_component_value('VHSin', frequencytoltpulse(fsw, high_side))
    netlist.set_component_value('VLSin', frequencytoltpulse(fsw, low_side))
    inductor = crs.random_coilcraft_inductor()
    print(inductor['PartNumber'])
    netlist.add_instruction(inductor['SubCkt'])
    # capacitor = mrs.random_murata_capacitor('C1')
    # print(capacitor['PartNumber'])
    # netlist.add_instruction(capacitor['SubCkt'])

    netlist.add_instruction(mrs.random_murata_capacitor('C1')['SubCkt'])
    netlist.add_instruction(mrs.random_murata_capacitor('C2')['SubCkt'])
    netlist.add_instruction(mrs.random_murata_capacitor('C3')['SubCkt'])
    netlist.add_instruction(mrs.random_murata_capacitor('C4')['SubCkt'])

    # print(netlist.get_components())

    netlist.write_netlist(Path("LTFiles/FullyCustom.net"))
    raw, log = LTC.run_now(netlist, run_filename="FullyCustom.net")

    LTR = RawRead('LTFiles/FullyCustom.raw')
    # print(LTR.get_trace_names())

    v_out = LTTraceData(LTR, 'V(vout)')
    print(v_out.min)
    print(v_out.max)
    print(v_out.average)
    v_out_max = v_out.max
    v_out_min = v_out.min

# print(raw)
# print(log)

# # run_netlist_file = f'LTFiles/F{fsw}_L{L}_C{C}'
# # run_netlist_file = f'LTFiles/FullyRandom'
# # LTC.run(run_filename=f'{run_netlist_file}.net')

SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started


OK
0805HS-151
Mon May 15 22:45:21 2023 info: : Starting simulation 1
Mon May 15 22:45:57 2023 info: Simulation Successful. Time elapsed: 00:00:00
Mon May 15 22:45:57 2023 info: Simulation Finished. Calling...dummy_callback(rawfile, logfile)
Reading the file with encoding  utf_16_le
File contains 39 traces, reading 39
Binary RAW file with Normal access


SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started


-8.118116
20.65945
4.284564
OK
1008LS-102
Mon May 15 22:46:00 2023 info: : Starting simulation 1
Mon May 15 22:47:41 2023 info: Simulation Successful. Time elapsed: 00:00:00
Mon May 15 22:47:41 2023 info: Simulation Finished. Calling...dummy_callback(rawfile, logfile)
Reading the file with encoding  utf_16_le
File contains 39 traces, reading 39
Binary RAW file with Normal access


SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started


-9.069148
20.805885
4.373059
OK
0402FL-561
Mon May 15 22:47:45 2023 info: : Starting simulation 1
Mon May 15 22:48:22 2023 info: Simulation Successful. Time elapsed: 00:00:00
Mon May 15 22:48:22 2023 info: Simulation Finished. Calling...dummy_callback(rawfile, logfile)
Reading the file with encoding  utf_16_le
File contains 39 traces, reading 39
Binary RAW file with Normal access


SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started


-10.148413
18.439207
3.9788954
OK
1008CS-392
Mon May 15 22:48:25 2023 info: : Starting simulation 1
Mon May 15 22:48:47 2023 info: Simulation Successful. Time elapsed: 00:00:00
Mon May 15 22:48:47 2023 info: Simulation Finished. Calling...dummy_callback(rawfile, logfile)
Reading the file with encoding  utf_16_le
File contains 39 traces, reading 39
Binary RAW file with Normal access


SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started


1.1461504
1.6495548
1.389297
OK
XAL7050-223
Mon May 15 22:48:49 2023 info: : Starting simulation 1
Mon May 15 22:49:09 2023 info: Simulation Successful. Time elapsed: 00:00:00
Mon May 15 22:49:09 2023 info: Simulation Finished. Calling...dummy_callback(rawfile, logfile)
Reading the file with encoding  utf_16_le
File contains 39 traces, reading 39
Binary RAW file with Normal access
4.497117
5.3707824
4.9392443
